# Analysis of HDFC Stock


Inspect the data frame

In [1]:
import pandas as pd

hdfc = pd.read_csv('HDFC.csv')
df = hdfc
df.head()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,HDFC,EQ,15-May-2017,1549.80,1554.50,1572.60,1554.10,1561.0,1559.50,1562.62,1270297,1.984990e+09,59917,955875,75.25
1,HDFC,W2,15-May-2017,204.45,217.95,217.95,205.00,205.0,205.00,209.50,29200,6.117400e+06,4,29200,100.00
2,HDFC,EQ,16-May-2017,1559.50,1558.00,1569.00,1554.00,1568.0,1566.55,1564.05,2114918,3.307844e+09,79354,1652379,78.13
3,HDFC,EQ,17-May-2017,1566.55,1565.50,1570.95,1549.75,1550.1,1552.50,1557.08,2161434,3.365526e+09,80317,1794472,83.02
4,HDFC,W2,17-May-2017,205.00,204.95,207.00,204.95,207.0,207.00,205.75,73000,1.501938e+07,10,73000,100.00


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  829 non-null    object 
 1   Series                  829 non-null    object 
 2   Date                    829 non-null    object 
 3   Prev Close              829 non-null    float64
 4   Open Price              829 non-null    float64
 5   High Price              829 non-null    float64
 6   Low Price               829 non-null    float64
 7   Last Price              829 non-null    float64
 8   Close Price             829 non-null    float64
 9   Average Price           829 non-null    float64
 10  Total Traded Quantity   829 non-null    int64  
 11  Turnover                829 non-null    float64
 12  No. of Trades           829 non-null    int64  
 13  Deliverable Qty         829 non-null    int64  
 14  % Dly Qt to Traded Qty  829 non-null    fl


Remove rows with non 'EQ' series.

In [3]:
df['Series'].unique()

array(['EQ', 'W2', 'BL'], dtype=object)

In [4]:
df = df[df['Series'] == 'EQ']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 828
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  494 non-null    object 
 1   Series                  494 non-null    object 
 2   Date                    494 non-null    object 
 3   Prev Close              494 non-null    float64
 4   Open Price              494 non-null    float64
 5   High Price              494 non-null    float64
 6   Low Price               494 non-null    float64
 7   Last Price              494 non-null    float64
 8   Close Price             494 non-null    float64
 9   Average Price           494 non-null    float64
 10  Total Traded Quantity   494 non-null    int64  
 11  Turnover                494 non-null    float64
 12  No. of Trades           494 non-null    int64  
 13  Deliverable Qty         494 non-null    int64  
 14  % Dly Qt to Traded Qty  494 non-null    fl

## Section 2
Analyse Close Prices

In [5]:
df['Close Price'].describe()

count     494.000000
mean     1823.898077
std       120.794912
min      1511.500000
25%      1731.425000
50%      1829.025000
75%      1929.187500
max      2069.150000
Name: Close Price, dtype: float64

## Section 3
Convert Date column to correct datatype

In [6]:
df['Date'].describe()

count             494
unique            494
top       08-Nov-2017
freq                1
Name: Date, dtype: object

In [7]:
df['Date'] = df['Date'].astype('datetime64')

## Section 4
Calculate VWAP

In [8]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [9]:
df.assign(PV = lambda x: (x['Average Price'] * x['No. of Trades']))\
    .groupby(['Year', 'Month'])\
    .sum()\
    .assign(VWAP = lambda x: x['PV']/x['No. of Trades'])['VWAP']

Year  Month
2017  5        1551.310538
      6        1632.487434
      7        1669.570491
      8        1744.280922
      9        1763.431565
      10       1728.358314
      11       1715.691379
      12       1694.765481
2018  1        1847.126268
      2        1826.146409
      3        1812.936182
      4        1844.511000
      5        1863.444751
      6        1858.946026
      7        1961.480250
      8        1944.221334
      9        1837.029787
      10       1717.892143
      11       1857.391383
      12       1944.531619
2019  1        1961.019706
      2        1907.079749
      3        1934.225716
      4        2006.661499
      5        1964.280996
Name: VWAP, dtype: float64


Calculate average and percent change of last N

In [10]:
def MovingAverageLastN(N: int, stock_df: pd.DataFrame) -> float:
    if N > stock_df.shape[0]:
        N = stock_df.shape[0]
    return stock_df['Close Price'].rolling(N).mean().iloc[-1]

In [11]:
def PLPercLastN(N: int, stock_df: pd.DataFrame) -> float:
    start = stock_df['Close Price'].iloc[1] if N >= stock_df.shape[0] else stock_df['Close Price'].iloc[-N]
    end = stock_df['Close Price'].iloc[-1]
    return (start - end)*100/end

In [12]:
calculations = {"Average Price": [MovingAverageLastN(n, df) for n in [7, 14, 30, 90, 180, 365]],\
                "P_L_Perc": [PLPercLastN(n, df) for n in [7, 14, 30, 90, 180, 365]]}

print(pd.DataFrame(calculations, index = ['1wk', '2wks', '1mth', '3mths', '6mths', '1yr/max']))

Average Price   P_L_Perc
1wk        1952.585714   2.739516
2wks       1963.007143   0.051206
1mth       1987.680000  -1.689795
3mths      1955.255556   0.791131
6mths      1904.292500  -0.583747
1yr/max    1870.354658 -12.668339



Calculate daywise percentage

In [13]:
df['Day_Perc_Change'] = df['Close Price'].pct_change() * 100

Remove first row

In [14]:
df.isna().any(axis=0)

Symbol                    False
Series                    False
Date                      False
Prev Close                False
Open Price                False
High Price                False
Low Price                 False
Last Price                False
Close Price               False
Average Price             False
Total Traded Quantity     False
Turnover                  False
No. of Trades             False
Deliverable Qty           False
% Dly Qt to Traded Qty    False
Month                     False
Year                      False
Day_Perc_Change            True
dtype: bool

The first value is NaN because on the first day has no percentage change

In [15]:
df.fillna(0, inplace = True)


Calculate and add 'Trends' column

In [16]:
bins = [-float("inf"), -7, -3, -1, -0.5, 0.5, 1, 3, 7, float("inf")]
labels = ['Bear drop', 'Among top losers', 'Negative', 'Slight negative', 'Slight or no change', 'Slight positive', 'Positive', 'Among top gainers', 'Bull run']
df['Trend'] = pd.cut(df['Day_Perc_Change'], bins = bins, right = False, labels = labels)

In [17]:
df['Trend'].unique()

[Slight or no change, Slight negative, Negative, Slight positive, Among top gainers, Positive, Among top losers]
Categories (7, object): [Among top losers < Negative < Slight negative < Slight or no change < Slight positive < Positive < Among top gainers]

Mean and Median Traded Quantity by Trend

In [18]:
Median_Traded_Quantity = df.groupby('Trend')['Total Traded Quantity'].median().dropna()
Median_Traded_Quantity

Trend
Among top losers       4736399.5
Negative               2807872.5
Slight negative        2224803.0
Slight or no change    2247984.0
Slight positive        2434008.5
Positive               2915465.5
Among top gainers      6450539.0
Name: Total Traded Quantity, dtype: float64

In [19]:
Mean_Traded_Quantity = df.groupby('Trend')['Total Traded Quantity'].mean().dropna()
Mean_Traded_Quantity

Trend
Among top losers       5.186927e+06
Negative               3.028490e+06
Slight negative        2.494395e+06
Slight or no change    2.506236e+06
Slight positive        2.583997e+06
Positive               3.075714e+06
Among top gainers      6.352677e+06
Name: Total Traded Quantity, dtype: float64

In [20]:
df.to_csv('week2.csv')